tgb - 3/2/2020 - The goal is to rederive the reduced data for the unstable LRFs using the same norm file that was used for the coupled runs.
- Define unstable NN
- Input-regularize base profile loaded from reduced data
- Calculate the corresponding LRFs
- Save the LRF(regularization_amplitude)  

# Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.utils import *
from cbrain.normalization import *
from cbrain.models import fc_model
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import pickle
import scipy.integrate as sin
from tensorflow.python.ops.parallel_for.gradients import batch_jacobian
%matplotlib inline

/home/t/Tom.Beucler/miniconda3/envs/cbrain_1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/envs/cbrain_1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/envs/cbrain_1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beuc

/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog


# Model definition

In [2]:
in_vars = ['QBP', 'TBP', 'VBP', 'PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ', 'TPHYSTND', 'FSNT', 'FSNS', 'FLNT', 'FLNS', 'PRECT']

In [3]:
scale_dict = load_pickle('/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/nn_config/scale_dicts/002_pnas_scaling.pkl')

In [4]:
input_transform = InputNormalizer(
    xr.open_dataset(f'/project/meteo/w2w/A6/S.Rasp/SP-CAM/preprocessed_data/003_norm.nc'),
    in_vars,
    'mean', 'maxrs')

In [5]:
output_transform = DictNormalizer(xr.open_dataset(f'/project/meteo/w2w/A6/S.Rasp/SP-CAM/preprocessed_data/003_norm.nc'), 
                                  out_vars, scale_dict)

In [6]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
def fc_model(input_shape, output_shape, hidden_layers, conservation_layer=False,
             inp_sub=None, inp_div=None, norm_q=None):
    inp = Input(shape=(input_shape,))

    # First hidden layer
    x = Dense(hidden_layers[0])(inp)
    x = LeakyReLU()(x)

    # Remaining hidden layers
    for h in hidden_layers[1:]:
        x = Dense(h)(x)
        x = LeakyReLU()(x)

    if conservation_layer:
        x = SurRadLayer(inp_sub, inp_div, norm_q)([inp, x])
        x = MassConsLayer(inp_sub, inp_div, norm_q)([inp, x])
        out = EntConsLayer(inp_sub, inp_div, norm_q)([inp, x])

    else:
        out = Dense(output_shape)(x)

    return tf.keras.models.Model(inp, out)

In [7]:
UNSTABNN_path = '/local/Tom.Beucler/SPCAM_PHYS/Noah32col_weights.h5'

unstabNN = fc_model(94, 65, [256]*9)
unstabNN.load_weights(f'{UNSTABNN_path}')

# Perturbation experiments

## Test reduced data

In [8]:
path = '/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/'
hf = open(path+'9_9_LRF.pkl','rb')

In [9]:
S = pickle.load(hf)

In [10]:
base_state = S['base_state'][1]

In [11]:
base_state.keys()

dict_keys(['qv', 'T', 'v', 'ps', 'S0', 'SHF', 'LHF', 'p', 'p_interface', 'z', 'z_interface', 'rho', 'rho_interface'])

In [12]:
S['base_state'].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## Perturb the data by 0,1,2.5,5,7.5,10,12.5,15,17.5,20,22.5,25% for Figures 10 and 11

In [13]:
def get_jacobian(x, model):
    sess = tf.keras.backend.get_session()
    jac = jacobian(model.output, model.input)
    J = sess.run(jac, feed_dict={model.input: x.astype(np.float32)[None]})
    return J.squeeze()
def get_batch_jacobian(x, model):
    sess = tf.keras.backend.get_session()
    jac = batch_jacobian(model.output, model.input)
    J = sess.run(jac, feed_dict={model.input: x.astype(np.float32)})
    return J.squeeze()

In [14]:
# Variables that we will perturb 
profiles = {'qv','T','v'}
scalars = {'ps','S0','SHF','LHF'}

In [15]:
cf_inp = np.zeros((1,94))
for index in range (94):
    if index<30: cf_inp[0,index]=L_V;
    elif index<60: cf_inp[0,index]=C_P;
    else: cf_inp[0,index]=1;
        
cf_oup = np.zeros((1,65))
for index in range (65):
    if index<30: cf_oup[0,index]=L_V;
    elif index<60: cf_oup[0,index]=C_P;
    else: cf_oup[0,index]=1;

In [16]:
it = 1
Name = ['MeanLRF_unstable']
Np = np.size(base_state['p']) # Number of vertical levels
Npert = 5000 # Number of perturbations
pert_array = 0.01*np.array([0,1,2.5,5,7.5,10,12.5,15,17.5,20,22.5,25])
pert_state = {}
jac = {}
pert_state_mean = {}

In [19]:
for ipert,pert in enumerate(pert_array):
    print('ipert=',ipert,' perturbation=',pert,'          ')
    # Perturbed profiles
    pert_state[ipert] = {};
    for i,profile in enumerate(profiles):
        print('i=',i,' profile=',profile,'          ',end='\r')
        pert_state[ipert][profile] = np.zeros((Np,Npert))
        for j,lev in enumerate(base_state['p']):
            pert_state[ipert][profile][j,:] = base_state[profile][j]+\
            np.random.normal(loc=0,scale=pert,size=(Npert,))*\
            np.tile(base_state[profile][j],(Npert,))
    # Perturbed scalars
    for i,scalar in enumerate(scalars):
        print('i=',i,' scalar=',scalar,'          ',end='\r')
        pert_state[ipert][scalar] = base_state[scalar]+\
        np.random.normal(loc=0,scale=pert,size=(Npert,))*\
        np.tile(base_state[scalar],(Npert,))
    # Perturbed input batch to feed to NN
    in_vec_pert = np.concatenate([pert_state[ipert]['qv'],
                                  pert_state[ipert]['T'],
                                  pert_state[ipert]['v'],
                                  np.tile(pert_state[ipert]['ps'],(1,1)),
                                  np.tile(pert_state[ipert]['S0'],(1,1)),
                                  np.tile(pert_state[ipert]['SHF'],(1,1)),
                                  np.tile(pert_state[ipert]['LHF'],(1,1)),
                                 ])[None, :].astype('float32')
    in_vec_pert = np.transpose(in_vec_pert[0,:,:]) # Shape = [#batches,input size]
    # Unstable Jacobian
    print('Calculating Jacobian from unstable NN')
    Junstab = get_batch_jacobian(input_transform.transform(in_vec_pert),unstabNN)*\
    np.transpose(cf_oup/output_transform.scale)/\
    (cf_inp*input_transform.div)
    # Linear response functions
    print('Linear response functions')
    LRFunstab = Junstab[:,:(2*Np),:(2*Np)] # Only keep the d(dq/dt,dT/dt)/d(q,T) Jacobian
    LRFunstab_mean = np.mean(LRFunstab,axis=0)
    # Save LRFs
    jac[ipert] = {}
    for ijac,name in enumerate(Name):
        if ijac==0: tmp = LRFunstab_mean
        jac[ipert][name] = {}
        jac[ipert][name]['q'] = {}
        jac[ipert][name]['T'] = {}
        jac[ipert][name]['q']['q'] = tmp[:30,:30]
        jac[ipert][name]['q']['T'] = tmp[:30,30:]
        jac[ipert][name]['T']['q'] = tmp[30:,:30]
        jac[ipert][name]['T']['T'] = tmp[30:,30:]
    # Calculate mean of pert_state which should be close to base_state for reference
    pert_state_mean[ipert] = {}
    for i,profile in enumerate(profiles):
        print('i=',i,' profile=',profile,'          ',end='\r')
        pert_state_mean[ipert][profile] = np.mean(pert_state[ipert][profile],axis=1)
    for i,scalar in enumerate(scalars):
        print('i=',i,' scalar=',scalar,'          ',end='\r')
        pert_state_mean[ipert][scalar] = np.mean(pert_state[ipert][scalar])


ipert= 0  perturbation= 0.0           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 1  perturbation= 0.01           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 2  perturbation= 0.025           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 3  perturbation= 0.05           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 4  perturbation= 0.075           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 5  perturbation= 0.1           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 6  perturbation= 0.125           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 7  perturbation= 0.15           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 8  perturbation= 0.17500000000000002           
Calculating Jacobian from unstable NN
Linear response functions
ipert= 9  perturbation= 0.2           
Calculati

In [20]:
hf = open(path+'2020_03_02_LRF_Unstable.pkl','wb')
ForNoah = {"base_state" : base_state,
           "mean_pert_state" : pert_state_mean,
           "linear_response_functions" : jac}
pickle.dump(ForNoah,hf)
hf.close()